# ModBus/RS485 registers definition helper

Purpose of this lib is to streamline management of the ModBus registers definitions by using common, compact, yet flexible, file format - CSV.

## Definition file format

The definition file:
1. Must have header row
1. Uses one row per ModBus register
1. Must include `address`, `name`, `type` and `unit` columns (required)
1. May include `divisor` or `multiplier` column
1. May define dictionaries using `dictName_fieldName` columns naming convention
1. May/should include description field(s) - defined as dictionary `desc` with locale specified as field name

## Predefined columns

|Column|Req.|Value|Description / Remarks|
|------|----|-----|-----------|
|`address`|+|`int`|Register number - base 10 (`1024`) or base 16 (`0x0400`)|
|`name`|+|`str`|Register name - 'machine friendly'; should not contain spaces|
|`type`|+|`Enum<type>[:len][:(Enum\|Flags)]`|Register type definition; may include optional length and subtype (`Enum`/`Flags`)|
|`unit`|+|`str\|dict`|Register unit - either unit desc string or dict defining `Enum`/`Flags` (required if such subtype is used)|
|`divisor`|-|`int`|Register value divisor, defaults to `1` - exclusive with `multiplier`|
|`groups`|-|`set`|Set of groups the register belongs to (tags of the register)|
|`multiplier`|-|`float`|Register value multiplier, defaults to `1` - exclusive with `divisor`, takes precedence|
|`desc.*`|-|`dict<str:str>`|Register description - human friendly, localized; keys are treated as locale codes, so use `dict.en` for `EN` desc, `dict.pl` for `PL` one and so forth|

## Register types

Register definition `type` column value is converted into `RegisterType` enum. This enum captures Python `struct` format char for each ModBus data type and adds utility methods for computing packed binary data size for the register of given `type` and length.

Following types can be used as `type` column values:

|Type value(s)|C type|`struct` fmt char|
|-------------|---------|-----------------|
| `I16`, `INT16` | `short` | `h` |
| `U16`, `UINT16` | `unsigned short` | `H` |
| `INT`, `I32`, `INT32` | `int` | `i` |
| `UINT`, `U32`, `UINT32` | `unsigned int` | `I` |
| `LONG`, `I64`, `INT64` | `long` | `l` |
| `U64`, `UINT64` | `unsigned long` | `L` |
| `FLOAT`, `F32`, `F` | `float` | `f` |
| `DOUBLE`, `F64`, `D` | `double` | `d` |
| `BITMAP`, `BITS` | `unsignet char` | `B` |
| `ASCII`, `CHAR` | `char[]` | `s` |

### Optional modifiers

The `type` column value may contain __optional modifiers__ - separated with `:`:
1. `len` - length of the register (length in bytes is given as: `struct.calcsize(fmt_char) * len`)
1. `subtype` - `Enum` or `Flags` - hinting that register values can be enumerated


## Enums/Flags

If `Enum` or `Flags` is specified as a field subtype, the `unit` column must contain dictionary specifying such `Enum`/`IntFlags` values, for example:
* given register with `name='SysState'` and `type` of `U16::Enum`:
  * when `unit` column contains:
    ```
    {
      0: Waiting
      1: Checking
      2: On-grid
      3: Emergency Mode
      ...
    }
    ```
  * then following Python `Enum` will be created and used in register definition:
    ```python
    Enum('SysState', [
      ('Waiting', 0),
      ('Checking', 1),
      ('On-grid', 2),
      ('Emergency Mode', 3),
      ...
    ])
    ```
  * which can be used as:
    ```python
    sys_state_def = registers['SysState']
    rs485_resp = get_registers(registers)
    print(f'SysState: {sys_state_def.unit(rs485_resp[sys_state_def.address]).name}')
    ```
* given register with `name='RemoteConfig'` and `type` of `BITMAP:2:Flags`:
  * when `unit` column contains:
    ```
    {
      0b00000001: AC power derate - active
      0b00000010: Remote on/off capability
      0b00000100: AC power derate - high volt.
      ...
    }
    ```
  * then following Python `IntFlag` will be created and used in register definition:
    ```python
    IntFlag('RemoteConfig', [
      ('AC power derate - active', 1),
      ('Remote on/off capability', 2),
      ('AC power derate - high volt.', 4),
      ...
    ])
    ```
      
### For example

Given `sofar.csv` file, following `Enum`'s and `IntFlag`'s are created when reading definitions:

In [5]:
import sys
import csv
from pprint import pp as pprint
from IPython.display import display, Markdown

import registers

with open('sofar.csv') as f:
    regs = registers.read(f)
    md = []
    for r in regs:
        if type(r.unit) is not str:
            md.append(f'* {r.name}: {r.type.name}')
            md += [f'  * {r.unit[n].value}: {n}' for n in r.unit.__members__]
    display(Markdown('\n'.join(md)))

* SysState: U16
  * 0: Waiting
  * 1: Checking
  * 2: On-grid
  * 3: Emergency Mode
  * 4: Recoverable Fault
  * 5: Permanent Fault
  * 6: Upgarding
  * 7: Self-charging
* VoltageConfig: BITMAP
  * 64: OVP - 10 min.
  * 32: UVP 3
  * 16: UVP 2
  * 8: UVP 1
  * 4: OVP 3
  * 2: OVP 2
  * 1: OVP 1
* RemoteConfig: BITMAP
  * 1: AC power derate - active
  * 2: Remote on/off capability
  * 4: AC power derate - high volt.
  * 8: Batt. charging power derate - high volt.
  * 16: Batt. charging power derate - low volt.
  * 32: DRM0
  * 64: DRM1-8
  * 128: Reflux power overload
* AntiReflux_Control: U16
  * 0: disabled
  * 1: enabled
  * 2: enabled - avg. power mode
* UnbalancedSupport_Control: U16
  * 0: disabled
  * 1: enabled

## Usage

Just import the `registers` package and use one of functions:
* `read(file, **fmtparams)` - parses CSV `file`; `fmtparams` are passed to the csv `DictReader`

In [6]:
import sys
import csv
from pprint import pp as pprint

import registers

## Exmple usage

Given CSV file (i.e. `sofar.csv`)

In [2]:
from enum import Enum
from IPython.display import display, Markdown

with open('sofar.csv') as f:
    regs = registers.read(f)
    md = """
|Addr|Name|Type|Unit|Desc|Groups|
|----|----|----|----|----|------|""".splitlines()
          
    for r in regs:
          md.append(f'|{r.address}|{r.name}|{r.type.name}|{r.unit.name if type(r.unit) is Enum else r.unit}|{r.description}|{r.groups}|')
          
    display(Markdown('\n'.join(md)))


|Addr|Name|Type|Unit|Desc|Groups|
|----|----|----|----|----|------|
|1028|SysState|U16|<enum 'SysState'>|System state|{'main', 'status'}|
|1048|Temperature_Env1|I16|degC|System temperature|{'main', 'status', 'temp'}|
|1049|Temperature_Env2|I16|degC|System temperature 2|{'status', 'temp'}|
|1029|Fault1|U16|||{'status', 'fault'}|
|1030|Fault2|U16|||{'status', 'fault'}|
|1031|Fault3|U16|||{'status', 'fault'}|
|5248|EvLst_ID1|U16|||{'status', 'event'}|
|5249|EvLst_YM1|U16|||{'status', 'event'}|
|5250|EvLst_DH1|U16|||{'status', 'event'}|
|5251|EvLst_MS1|U16|||{'status', 'event'}|
|5252|EvLst_ID2|U16|||{'status', 'event'}|
|5253|EvLst_YM2|U16|||{'status', 'event'}|
|5254|EvLst_DH2|U16|||{'status', 'event'}|
|5255|EvLst_MS2|U16|||{'status', 'event'}|
|1156|Frequency_Grid|U16|Hz|Grid frequency|{'main', 'metrics', 'grid'}|
|1157|ActivePower_Output_Total|I16|kW||{'main', 'power', 'metrics', 'grid'}|
|1159|ApparentPower_Output_Total|I16|kW||{'power', 'metrics', 'apparent', 'grid'}|
|1160|ActivePower_PCC_Total|I16|kW||{'main', 'power', 'metrics', 'pcc'}|
|1162|ApparentPower_PCC_Total|I16|kW||{'power', 'metrics', 'apparent', 'pcc'}|
|1284|ActivePower_Load_Total|I16|kW||{'main', 'power', 'metrics', 'load'}|
|1286|ApparentPower_Load_Total|I16|kW||{'power', 'metrics', 'load', 'apparent'}|
|1165|Voltage_Phase_R|U16|V|Grid phase R voltage|{'voltage', 'metrics', 'line'}|
|1166|Current_Output_R|U16|A||{'current', 'metrics', 'line'}|
|1176|Voltage_Phase_S|U16|V|Grid phase S voltage|{'voltage', 'metrics', 'line'}|
|1177|Current_Output_S|U16|A||{'current', 'metrics', 'line'}|
|1187|Voltage_Phase_T|U16|V|Grid phase T voltage|{'voltage', 'metrics', 'line'}|
|1188|Current_Output_T|U16|A||{'current', 'metrics', 'line'}|
|1198|ActivePower_PV_Ext|U16|kW||{'power', 'metrics', 'pv'}|
|1199|ActivePower_Load_Sys|U16|kW||{'power', 'metrics', 'load', 'sys'}|
|1412|Voltage_PV1|U16|V|PV MPPT 1 voltage|{'voltage', 'metrics', 'pv'}|
|1413|Current_PV1|U16|A|PV MPPT 1 current|{'current', 'metrics', 'pv'}|
|1414|Power_PV1|U16|kW|PV MPPT 1 power|{'main', 'power', 'metrics', 'pv'}|
|1415|Voltage_PV2|U16|V|PV MPPT 2 voltage|{'voltage', 'metrics', 'pv'}|
|1416|Current_PV2|U16|A|PV MPPT 2 current|{'current', 'metrics', 'pv'}|
|1417|Power_PV2|U16|kW|PV MPPT 2 power|{'main', 'power', 'metrics', 'pv'}|
|1540|Voltage_Bat1|U16|V|Battery 1 voltage|{'bat', 'voltage', 'metrics'}|
|1541|Current_Bat1|I16|A|Battery 1 current|{'bat', 'current', 'metrics'}|
|1542|Power_Bat1|I16|kW|Battery 1 power|{'bat', 'power', 'metrics'}|
|1546|ChargeCycle_Bat1|U16|cycle|Battery 1 cycles|{'bat', 'metrics', 'cycle'}|
|1547|Voltage_Bat2|U16|V|Battery 2 voltage|{'bat', 'voltage', 'metrics'}|
|1548|Current_Bat2|I16|A|Battery 2 current|{'bat', 'current', 'metrics'}|
|1549|Power_Bat2|I16|kW|Battery 2 power|{'bat', 'power', 'metrics'}|
|1553|ChargeCycle_Bat2|U16|cycle|Battery 2 cycles|{'bat', 'metrics', 'cycle'}|
|1924|ArcStrength_Chn1|I16|||{'arc', 'metrics'}|
|1925|ArcStrength_Chn2|I16|||{'arc', 'metrics'}|
|8196|Country_Code|U16||Safety settings - Country code|{'info'}|
|8199|Safety_Name|ASCII||Safety settings - name|{'info'}|
|2112|VoltageConfig|BITMAP|<enum 'VoltageConfig'>|Used grid protection parameters|{'ovp', 'info'}|
|2113|RatedVoltage|U16|V||{'info'}|
|2114|FirstOvervoltageProtectionValue|U16|V||{'ovp', 'info'}|
|2115|FirstOvervoltageProtectionTime|U16|10ms||{'ovp', 'info'}|
|2116|SecondOvervoltageProtectionValue|U16|V||{'ovp', 'info'}|
|2117|SecondOvervoltageProtectionTime|U16|10ms||{'ovp', 'info'}|
|2118|ThirdOvervoltageProtectionValue|U16|V||{'ovp', 'info'}|
|2119|ThirdOvervoltageProtectionTime|U16|10ms||{'ovp', 'info'}|
|2120|FirstUnderVoltageProtectionValue|U16|V||{'ovp', 'info'}|
|2121|FirstUndervoltageProtectionTime|U16|10ms||{'ovp', 'info'}|
|2122|SecondUnderVoltageProtectionValue|U16|V||{'uvp', 'info'}|
|2123|SecondUndervoltageProtectionTime|U16|10ms||{'uvp', 'info'}|
|2124|ThirdUnderVoltageProtectionValue|U16|V||{'uvp', 'info'}|
|2125|ThirdUndervoltageProtectionTime|U16|10ms||{'uvp', 'info'}|
|2126|10MinOvervoltageProtectionValue|U16|V||{'ovp', 'info'}|
|2304|RemoteConfig|BITMAP|<enum 'RemoteConfig'>||{'info'}|
|2305|ActiveOutputLimit|U16|%||{'info'}|
|2306|ActiveOutputDownSpeed|U16|%Pn/min||{'info'}|
|2307|GridVoltageDropStart|U16|V||{'info'}|
|2308|GridVoltageDropStop|U16|V||{'info'}|
|2309|GridVoltageDropMinPower|I16|%||{'info'}|
|2310|OvervoltageDownSpeed|U16|%Pn/min||{'info'}|
|2322|RefluxPower|U16|%||{'info'}|
|2323|RefluxOVloadTime|U16|10ms||{'info'}|
|4131|AntiReflux_Control|U16|<enum 'AntiReflux_Control'>||{'status'}|
|4132|AntiReflux_Power|U16|10W||{'status'}|
|4152|UnbalancedSupport_Control|U16|<enum 'UnbalancedSupport_Control'>||{'status'}|

## User-defined fields and dictionaries

Any non-predefined column from the CSV file will translate to additional field in returned `@dataclass` (it's defined dynamically).

Dot-notation can be used to group columns; i.e.: 
* columns `ha.icon` and `ha.name` will be placed in `ha` field 
* field type will be dynamically defined `@dataclass`:
    ```python
    @dataclass HaRec:
        icon,
        name
    ```

For example:

In [12]:
import sys
import csv
from pprint import pp as pprint

import registers

from IPython.display import display, Markdown
csv = """address,name,desc.en,desc.pl,type,divisor,unit,groups,ha.icon,ha.name,my_field
         0x0418,Temperature_Env1,System temperature,Temperatura falownika,I16,1,degC,"{status,temp,main}",mdi:temperature,Inverter temperature,General info
         0x0485,ActivePower_Output_Total,,,I16,100,kW,"{metrics,grid,power,main}",mdi:,Total output power - sent to grid,General info
         0x0405,Fault1,,,U16,,,"{status,fault}",mdi:error,Inverter - last fault code,General info
         """

regs = registers.read(csv.splitlines())
pprint(regs)

[ModbusRegister(address=1048, name='Temperature_Env1', type=<RegisterType.I16: 1>, len=1, unit='degC', groups={'main', 'status', 'temp'}, divisor='1', desc={'en_US.ISO8859-1': 'System temperature', 'pl_PL.ISO8859-2': 'Temperatura falownika'}, ha=HaRec(icon='mdi:temperature', name='Inverter temperature'), my_field='General info'),
 ModbusRegister(address=1157, name='ActivePower_Output_Total', type=<RegisterType.I16: 1>, len=1, unit='kW', groups={'main', 'power', 'metrics', 'grid'}, divisor='100', desc={'en_US.ISO8859-1': '', 'pl_PL.ISO8859-2': ''}, ha=HaRec(icon='mdi:', name='Total output power - sent to grid'), my_field='General info'),
 ModbusRegister(address=1029, name='Fault1', type=<RegisterType.U16: 2>, len=1, unit='', groups={'status', 'fault'}, divisor='', desc={'en_US.ISO8859-1': '', 'pl_PL.ISO8859-2': ''}, ha=HaRec(icon='mdi:error', name='Inverter - last fault code'), my_field='General info')]
